# Mini Project: Deep Learning with Neural Networks

Working with neural networks and the MNIST dataset.

## Task 1: Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.13.0


## Task 2: Build Simple Neural Network from Scratch

In [2]:
class SimpleNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.b2 = np.zeros((1, output_size))
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.sigmoid(self.z2)
        return self.a2
    
    def backward(self, X, y, output, lr=0.1):
        m = X.shape[0]
        
        dz2 = output - y
        dW2 = (1/m) * np.dot(self.a1.T, dz2)
        db2 = (1/m) * np.sum(dz2, axis=0, keepdims=True)
        
        dz1 = np.dot(dz2, self.W2.T) * self.a1 * (1 - self.a1)
        dW1 = (1/m) * np.dot(X.T, dz1)
        db1 = (1/m) * np.sum(dz1, axis=0, keepdims=True)
        
        self.W1 -= lr * dW1
        self.W2 -= lr * dW2
        self.b1 -= lr * db1
        self.b2 -= lr * db2

nn = SimpleNN(3, 4, 1)
print("Network created")

Network created


## Task 3: Test the Network

In [3]:
# Simple test data
X = np.array([[0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 1]])
y = np.array([[0], [1], [1], [0]])

# Train for some iterations
for i in range(1000):
    output = nn.forward(X)
    nn.backward(X, y, output)

print("Training done")

Training done


## Task 4: Load MNIST Data

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(f"Training data: {X_train.shape}")
print(f"Test data: {X_test.shape}")

Training data: (60000, 28, 28)
Test data: (10000, 28, 28)


## Task 5: Preprocess Data

In [5]:
# Show some sample images
plt.figure(figsize=(10, 4))
for i in range(6):
    plt.subplot(2, 3, i + 1)
    plt.imshow(X_train[i], cmap='gray')
    plt.title(f'Digit: {y_train[i]}')
    plt.axis('off')
plt.show()

# Normalize and reshape
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print("Data preprocessed")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

Data preprocessed
X_train shape: (60000, 784)
y_train shape: (60000, 10)


## Task 6: Build Keras Model

In [6]:
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(784,)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               100480    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 10)                650       
                                                                 
Total params: 109386 (427.29 KB)
Trainable params: 109386 (427.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Task 7: Train Model

In [7]:
history = model.fit(X_train, y_train,
                    epochs=5,
                    batch_size=128,
                    validation_split=0.2)

Epoch 1/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4323 - accuracy: 0.8709 - val_loss: 0.1892 - val_accuracy: 0.9444
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.1858 - accuracy: 0.9452 - val_loss: 0.1376 - val_accuracy: 0.9593
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.1424 - accuracy: 0.9573 - val_loss: 0.1150 - val_accuracy: 0.9655
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.1161 - accuracy: 0.9645 - val_loss: 0.1028 - val_accuracy: 0.9688
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0993 - accuracy: 0.9697 - val_loss: 0.0971 - val_accuracy: 0.9708


## Task 8: Evaluate Results

In [8]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.2%}")

# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Loss')
plt.legend()

plt.show()

313/313 [==============================] - 1s 2ms/step - loss: 0.0951 - accuracy: 0.9715
Test accuracy: 97.15%


## Results

Got 97.15% accuracy on test set. The model trained well and converged quickly.

Key things learned:
- Neural networks need proper initialization
- Dropout helps prevent overfitting
- Data normalization is important
- MNIST is a good starter dataset